# 0. Install and Import Dependencies

In [ ]:
!pip install tensorflow==2.4.1 tensorflow-gpu==2.4.1 opencv-python matplotlib

In [11]:
import tensorflow as tf
import numpy as np
from matplotlib import pyplot as plt
import cv2
import time

# 1. Load Model

17개의 keypoints

In [2]:
interpreter = tf.lite.Interpreter(model_path='./lite-model_movenet_singlepose_lightning_3.tflite')
interpreter.allocate_tensors()

# 2. Make Detections

keypoint ->>> [nose, left eye, right eye, left ear, right ear, left shoulder, right shoulder, left elbow, right elbow, left wrist, right wrist, left hip, right hip, left knee, right knee, left ankle, right ankle]

Input_size = 192,192,3

In [12]:
def image():
    time.sleep(3)
    cap = cv2.VideoCapture(0)
    good=[]
    good=0
    less=[]
    less=0
    more=[]
    more=0
    # Curl counter variables
    counter = 0 
    stage = None

    while cap.isOpened():
        ret, frame = cap.read()

        # Reshape image
        img = frame.copy()
        img = tf.image.resize_with_pad(np.expand_dims(img,axis=0),192,192)
        input_image = tf.cast(img,dtype=tf.float32)

        # Setup input and output 
        input_details = interpreter.get_input_details()
        output_details = interpreter.get_output_details()

        # Make predictions 
        interpreter.set_tensor(input_details[0]['index'], np.array(input_image))
        interpreter.invoke()
        keypoints_with_scores = interpreter.get_tensor(output_details[0]['index'])

    #     print(keypoints_with_scores)
        ### keypoints - 17개

        # Rendering
        draw_connections(frame, keypoints_with_scores, EDGES, 0.4)
        draw_keypoints(frame, keypoints_with_scores, 0.4)


        # Extract landmarks
        try:
            right_shoulder = keypoints_with_scores[0][0][6][:2]
            right_elbow = keypoints_with_scores[0][0][8][:2]
            right_wrist = keypoints_with_scores[0][0][10][:2]

    #         np.array(left_elbow[:2]*[480,640]).astype(int)

            # Calculate angle
            angle = calculate_angle(right_shoulder, right_elbow, right_wrist)

            # Visualize angle
            cv2.putText(frame, str(angle), 
                           tuple(np.multiply(right_elbow, [480,640]).astype(int)), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA
                                )

            # Curl counter logic     

            if angle<=90:
                stage = "up"

            if angle >150 and stage =='up':
                stage="down"

                counter +=1
#                 print(counter)

        except:
            pass

        cv2.rectangle(frame, (0,0), (225,73), (245,117,16),-1)

        # Rep data
        cv2.putText(frame, 'REPS', (15,12), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
        cv2.putText(frame, str(counter), 
                    (10,60), 
                    cv2.FONT_HERSHEY_SIMPLEX, 2, (255,255,255), 2, cv2.LINE_AA)

        # Stage data
        cv2.putText(frame, 'STAGE', (65,12), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
        cv2.putText(frame, stage, 
                    (60,60), 
                    cv2.FONT_HERSHEY_SIMPLEX, 2, (255,255,255), 2, cv2.LINE_AA)


        cv2.imshow('Movenet Lightning', frame)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    
    cap.release()
    cv2.destroyAllWindows()
    return counter

In [13]:
image()

2

In [73]:
cap = cv2.VideoCapture(0)
good=[]
good=0
less=[]
less=0
more=[]
more=0
# Curl counter variables
counter = 0 
stage = None

while cap.isOpened():
    ret, frame = cap.read()
    
    # Reshape image
    img = frame.copy()
    img = tf.image.resize_with_pad(np.expand_dims(img,axis=0),192,192)
    input_image = tf.cast(img,dtype=tf.float32)
    
    # Setup input and output 
    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()
    
    # Make predictions 
    interpreter.set_tensor(input_details[0]['index'], np.array(input_image))
    interpreter.invoke()
    keypoints_with_scores = interpreter.get_tensor(output_details[0]['index'])

#     print(keypoints_with_scores)
    ### keypoints - 17개
    
    # Rendering
    draw_connections(frame, keypoints_with_scores, EDGES, 0.4)
    draw_keypoints(frame, keypoints_with_scores, 0.4)
    
    
    # Extract landmarks
    try:
        right_shoulder = keypoints_with_scores[0][0][6][:2]
        right_elbow = keypoints_with_scores[0][0][8][:2]
        right_wrist = keypoints_with_scores[0][0][10][:2]
        
#         np.array(left_elbow[:2]*[480,640]).astype(int)
        
        # Calculate angle
        angle = calculate_angle(right_shoulder, right_elbow, right_wrist)
        
        # Visualize angle
        cv2.putText(frame, str(angle), 
                       tuple(np.multiply(right_elbow, [480,640]).astype(int)), 
                       cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA
                            )

        # Curl counter logic     
                
        if angle<=90:
            stage = "up"
            if 85<=angle<=90:
                good +=1
            else:
                more+=1
        if angle >150 and stage =='up':
            stage="down"
            
            counter +=1
            print(counter)

    except:
        pass
    
    

    cv2.rectangle(frame, (0,0), (225,73), (245,117,16),-1)
    
    # Rep data
    cv2.putText(frame, 'REPS', (15,12), 
                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
    cv2.putText(frame, str(counter), 
                (10,60), 
                cv2.FONT_HERSHEY_SIMPLEX, 2, (255,255,255), 2, cv2.LINE_AA)

    # Stage data
    cv2.putText(frame, 'STAGE', (65,12), 
                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
    cv2.putText(frame, stage, 
                (60,60), 
                cv2.FONT_HERSHEY_SIMPLEX, 2, (255,255,255), 2, cv2.LINE_AA)


    cv2.imshow('Movenet Lightning', frame)
    
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break
        
cap.release()
cv2.destroyAllWindows()

# 3. Draw Keypoints

In [3]:
def draw_keypoints(frame, keypoints, confidence_threshold):
    y, x, c = frame.shape
    shaped = np.squeeze(np.multiply(keypoints, [y,x,1]))
    
    for kp in shaped:
        ky, kx, kp_conf = kp
        if kp_conf > confidence_threshold:
            cv2.circle(frame, (int(kx), int(ky)), 4, (0,255,0), -1) 

# 4. Draw Edges

In [4]:
EDGES = {
    (0, 1): 'm',
    (0, 2): 'c',
    (1, 3): 'm',
    (2, 4): 'c',
    (0, 5): 'm',
    (0, 6): 'c',
    (5, 7): 'm',
    (7, 9): 'm',
    (6, 8): 'c',
    (8, 10): 'c',
    (5, 6): 'y',
    (5, 11): 'm',
    (6, 12): 'c',
    (11, 12): 'y',
    (11, 13): 'm',
    (13, 15): 'm',
    (12, 14): 'c',
    (14, 16): 'c'
}

In [5]:
def draw_connections(frame, keypoints, edges, confidence_threshold):
    y, x, c = frame.shape
    shaped = np.squeeze(np.multiply(keypoints, [y,x,1]))
    
    for edge, color in edges.items():
        p1, p2 = edge
        y1, x1, c1 = shaped[p1]
        y2, x2, c2 = shaped[p2]
        
        if (c1 > confidence_threshold) & (c2 > confidence_threshold):      
            cv2.line(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0,0,255), 2)

In [6]:
def calculate_angle(a,b,c):
    a = np.array(a) # First
    b = np.array(b) # Mid
    c = np.array(c) # End
    
    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians*180.0/np.pi)
    
    if angle >180.0:
        angle = 360-angle
        
    return angle 